<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [3]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

--2025-11-07 13:14:03--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2025-11-07 13:14:04--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 4.228.31.150
Connecting to github.com (github.com)|4.228.31.150|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2025-11-07 13:14:04--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.

In [4]:
# Posibles bandas
os.listdir("./songs_dataset/")

['paul-simon.txt',
 'leonard-cohen.txt',
 'jimi-hendrix.txt',
 'beatles.txt',
 'lin-manuel-miranda.txt',
 'lorde.txt',
 'adele.txt',
 'joni-mitchell.txt',
 'missy-elliott.txt',
 'lil-wayne.txt',
 'bob-dylan.txt',
 'disney.txt',
 'nirvana.txt',
 'nursery_rhymes.txt',
 'dickinson.txt',
 'dr-seuss.txt',
 'alicia-keys.txt',
 'blink-182.txt',
 'patti-smith.txt',
 'amy-winehouse.txt',
 'radiohead.txt',
 'bruce-springsteen.txt',
 'bjork.txt',
 'lady-gaga.txt',
 'dolly-parton.txt',
 'rihanna.txt',
 'michael-jackson.txt',
 'prince.txt',
 'al-green.txt',
 'bieber.txt',
 'cake.txt',
 'ludacris.txt',
 'notorious_big.txt',
 'janisjoplin.txt',
 'dj-khaled.txt',
 'kanye-west.txt',
 'johnny-cash.txt',
 'eminem.txt',
 'kanye.txt',
 'nicki-minaj.txt',
 'Lil_Wayne.txt',
 'bruno-mars.txt',
 'Kanye_West.txt',
 'notorious-big.txt',
 'nickelback.txt',
 'britney-spears.txt',
 'drake.txt',
 'r-kelly.txt',
 'bob-marley.txt']

In [6]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/lady-gaga.txt', sep='/n', header=None)
df.head()

/tmp/ipykernel_160475/4107314280.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/lady-gaga.txt', sep='/n', header=None)


,0
0,"I'll undress you, 'cause you're tired"
1,Cover you as you desire
2,When you fall asleep inside my arms
3,May not have the fancy things
4,But I'll give you everything


In [7]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 3807


### 1 - Preprocesamiento

In [9]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

2025-11-07 13:18:13.467006: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-07 13:18:13.530559: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-07 13:18:14.675803: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [10]:
# Demos un vistazo
sentence_tokens[:2]

[["i'll", 'undress', 'you', "'cause", "you're", 'tired'],
 ['cover', 'you', 'as', 'you', 'desire']]

### 2 - Crear los vectores (word2vec)

In [11]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [12]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [13]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [14]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 3807


In [15]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 722


### 3 - Entrenar embeddings

In [16]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 225482.796875
Loss after epoch 1: 155035.328125
Loss after epoch 2: 152820.625
Loss after epoch 3: 148431.625
Loss after epoch 4: 138253.5
Loss after epoch 5: 131583.375
Loss after epoch 6: 122535.75
Loss after epoch 7: 108953.25
Loss after epoch 8: 104393.75
Loss after epoch 9: 101013.25
Loss after epoch 10: 98637.125
Loss after epoch 11: 96883.375
Loss after epoch 12: 95152.0
Loss after epoch 13: 93183.25
Loss after epoch 14: 92545.125
Loss after epoch 15: 91719.625
Loss after epoch 16: 91131.625
Loss after epoch 17: 87097.625
Loss after epoch 18: 82691.0
Loss after epoch 19: 81990.25


(372010, 606000)

### 4 - Ensayar

In [19]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["pray"], topn=10)

[('tap', 0.9334839582443237),
 ('bow', 0.9194691181182861),
 ('goes', 0.8999143242835999),
 ('street', 0.8603769540786743),
 ('those', 0.8600791096687317),
 ('chase', 0.8560516238212585),
 ('lonely', 0.848296582698822),
 ('listen', 0.826325535774231),
 ('burn', 0.8195632696151733),
 ('bring', 0.8118451237678528)]

In [20]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('she', -0.1458824872970581),
 ('his', -0.157462939620018),
 ('to', -0.16939155757427216),
 ('alejandro', -0.17621156573295593),
 ("she's", -0.17670799791812897),
 ('heart', -0.17988987267017365),
 ('floor', -0.18076960742473602),
 ('nobody', -0.186057910323143),
 ('make', -0.18935710191726685),
 ('so', -0.1907348930835724)]

In [24]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["lord"], topn=10)

[('fangs', 0.9204478859901428),
 ('teeth', 0.8739274740219116),
 ('open', 0.8721127510070801),
 ('show', 0.8357691764831543),
 ('mouth', 0.8266028165817261),
 ('quit', 0.7868215441703796),
 ('end', 0.7769829630851746),
 ('round', 0.7709757089614868),
 ('truth', 0.768057107925415),
 ('lovers', 0.7626691460609436)]

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('fancy', 0.8555483818054199),
 ('aha', 0.8468345999717712),
 ('eh', 0.8340397477149963),
 ("that's", 0.8336365222930908),
 ('faster', 0.8172706961631775)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

In [26]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.02308867  0.4989994  -0.09772125  0.3247849   0.17555384  0.07140884
  0.4374889   0.12166818 -0.2540025  -0.11188604  0.1843571  -0.00847309
  0.10333195  0.04072281 -0.28295925  0.01112818  0.01088119  0.27480528
 -0.00107298 -0.1688643  -0.11177701 -0.17242876  0.13007943  0.42336705
 -0.15853404  0.02897894 -0.17196481 -0.13828687  0.05504058 -0.04482167
  0.03885011 -0.19878817  0.13278128  0.2271028  -0.22426909  0.26837105
  0.14764781 -0.1059263  -0.24700642  0.11171305 -0.20681983 -0.12497331
  0.16483329  0.22778313  0.273591    0.09938694 -0.02638428 -0.11866252
  0.05156144  0.37531793 -0.04071515  0.01639819  0.04541741  0.09187078
 -0.05626888 -0.11509284 -0.43191615  0.06140081  0.02493665 -0.10181837
 -0.03297804 -0.28322178  0.1128493  -0.21662079  0.00569549 -0.24158387
  0.36527023  0.07453392 -0.02212254 -0.20098059  0.0801177   0.21510217
  0.04860479  0.5812557   0.07561997  0.06853098 -0.03368496 -0.21928751
  0.2944462   0.3886212  -0.2769008  -0.16999595 -0

In [27]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 0.9999999403953552),
 ("doin'", 0.7452163696289062),
 ('game', 0.7287333607673645),
 ('play', 0.6739518642425537),
 ("wasn't", 0.6699321866035461),
 ('denim', 0.6677326560020447),
 ('fame', 0.6619865894317627),
 ('start', 0.6596845388412476),
 ('yourself', 0.6507260203361511),
 ('whoa', 0.6472256779670715)]

In [28]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[("doin'", 0.7452163696289062),
 ('game', 0.7287333011627197),
 ('play', 0.6739518642425537),
 ("wasn't", 0.6699321269989014),
 ('denim', 0.6677326560020447),
 ('fame', 0.6619865894317627),
 ('start', 0.6596844792366028),
 ('yourself', 0.6507260203361511),
 ('whoa', 0.6472256779670715),
 ('song', 0.6442990303039551)]

### 5 - Visualizar agrupación de vectores

In [29]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
# Usar matplotlib como alternativa si plotly tiene problemas
plt.figure(figsize=(12, 8))
plt.scatter(vecs[:MAX_WORDS,0], vecs[:MAX_WORDS,1], alpha=0.5)
for i, label in enumerate(labels[:MAX_WORDS]):
    plt.annotate(label, (vecs[i,0], vecs[i,1]), fontsize=8, alpha=0.7)
plt.title('Word Embeddings 2D Visualization')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

# Usar matplotlib para 3D visualization
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(vecs[:MAX_WORDS,0], vecs[:MAX_WORDS,1], vecs[:MAX_WORDS,2], alpha=0.5)
ax.set_title('Word Embeddings 3D Visualization')
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')
ax.set_zlabel('Dimension 3')
plt.show()

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.